In [ ]:
# dataframe operations
import pandas as pd

# change sys path to access /scripts
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts')))

# utilities to get blob paths
from data.data_loader import get_blob_path_for_row

# load existing parquet files
df_pre2010 = pd.read_parquet('../data/raw/loc/veterans_history_project_resources_pre2010.parquet')
df_audio_analysis = pd.read_parquet('../learnings/audio_quality_analysis/audio_quality_analysis.parquet')

# check audio_id column type to match azure_blob_index type
df_audio_analysis['audio_id'] = df_audio_analysis['audio_id'].astype(df_pre2010['azure_blob_index'].dtype)

df_audio_analysis.head()

In [ ]:
# merge audio analysis with df_pre2010 as audio_id = azure_blob_index
df_merged = df_pre2010.merge(df_audio_analysis, left_on='azure_blob_index', right_on='audio_id', how='left')

# construct blob_path column
df_merged['blob_path'] = [get_blob_path_for_row(row, idx) for idx, row in df_merged.iterrows()]

# flatten column values in df_merged['blob_path'] for list with only one element
df_merged['blob_path'] = df_merged['blob_path'].apply(lambda x: x[0] if isinstance(x, list) and len(x) == 1 else None)
df_merged['blob_path'] = df_merged['blob_path'].astype(str)
df_merged.head()

In [ ]:
df_audio_analysis.columns

In [ ]:
# remove unnecessary columns
cols_to_remove_df_audio_analysis = ['audio_id', 'status', 'error_message']
df_merged = df_merged.drop(columns=cols_to_remove_df_audio_analysis)

In [ ]:
df_pre2010.columns

In [ ]:
# remove unnecessary columns
cols_to_remove_df_pre2010 = ['fulltext_file_url', 'title', 'description', 'partof', 'subject']
df_merged = df_merged.drop(columns=cols_to_remove_df_pre2010)

In [ ]:
df_merged.head()

In [ ]:
df_merged.to_parquet('../data/raw/loc/veterans_history_project_resources_pre2010_with_audio_features.parquet', index=False)